# Qubo

## Chemistry Example

In [0]:
# import stuff
from collections import OrderedDict
import itertools
import numpy as np
from pprint import pprint
from dadk.FujitsuEmulatedDASolver import *
emda_solver = FujitsuEmulatedDASolver()

number of GPU available: 0


In [0]:
# set up nested (ordered) Dict

# custom class
class IndexingArray(OrderedDict):
    def __missing__(self, key):
        val = self[key] = IndexingArray()
        return val
    
groups = {'H':1, 'CH3':15, 'CF3':69, 'OH':17, 'NH2':16, 'CH2CH3':29}
# groups = {'H':1, 'CH3':1, 'CF3':2, 'OH':1, 'NH2':1, 'CH2CH3':2}

R0 = np.array(['H', 'CH3', 'CF3', 'OH'])      # R1
R1 = np.array(['H', 'CH3', 'NH2', 'CH2CH3'])  # R2
R2 = np.array(['H', 'CH3', 'CF3', 'OH'])      # R3 
R3 = np.array(['H', 'CH3', 'NH2', 'CH2CH3'])  # R4
items = [R0, R1, R2, R3]
    
# populate dict
d = IndexingArray()
k=0
for i, r in enumerate([R0, R1, R2, R3]):
    for j, g in enumerate(r):
        d['R'+str(i)][g] = k
        k += 1
        
pprint(d)
pprint(d['R0']['CF3'])

IndexingArray([('R0',
                IndexingArray([('H', 0), ('CH3', 1), ('CF3', 2), ('OH', 3)])),
               ('R1',
                IndexingArray([('H', 4),
                               ('CH3', 5),
                               ('NH2', 6),
                               ('CH2CH3', 7)])),
               ('R2',
                IndexingArray([('H', 8), ('CH3', 9), ('CF3', 10), ('OH', 11)])),
               ('R3',
                IndexingArray([('H', 12),
                               ('CH3', 13),
                               ('NH2', 14),
                               ('CH2CH3', 15)]))])
2


In [0]:
# Objective function: max x1 + x2 + x3 + x4
def H_obj(d):
    H = BinPol()
    for k, v in d.items():
        for group, idx in v.items():
            H.set_term(groups[group], (idx,))
    return H

# print(H_obj(d))

In [0]:
# Constraint A: select exactly one item -> P(1-x-y+2xy)
def H_c1(d, P, r):
    p = BinPol().set_term(1,())
    list = [(group, idx) for group, idx in d[r].items()]
    for pair in itertools.combinations(list, r=2):
        p.set_term(-1,(pair[0][1],))
        p.set_term(-1,(pair[1][1],))
    p.multiply(p)
    p.multiply_scalar(P) 
    return p

# print(H_c1(d, r='R0', P=1))

In [0]:
# Constraint B: Ri and Rj cannot have the same group -> P(xy)
def H_c2(d, P, ri, rj):
    p = BinPol()
    list1 = [(group, idx) for group, idx in d[ri].items()]
    list2 = [(group, idx) for group, idx in d[rj].items()]
    for pair1 in list1:
        for pair2 in list2:
            if pair1[0] == pair2[0]:
                p.set_term(1, (pair1[1], pair2[1]))
    p.multiply_scalar(P) 
    return p
                           
# print(H_c2(d, ri='R0', rj='R1', P=1))

In [0]:
# Constraint C: g can be selected at most once (i.e. x + y <= 0) -> P(xy)
def H_c3(d, P, g, ri, rj):
    p = BinPol()
    p.set_term(1, (d[ri][g], d[rj][g]))
    p.multiply_scalar(4*P) 
    return p

# print(H_c3(d, P=1, g='CF3', ri='R0', rj='R2'))

In [0]:
# combine
def H_all(d, P):
    """Note how all constraints are multiplied by -1 to subtract instead of adding"""
    H = BinPol()
    H.add(H_obj(d))
    H.add(H_c1(d, P, r='R0').multiply_scalar(-1)) 
    H.add(H_c1(d, P, r='R1').multiply_scalar(-1))
    H.add(H_c1(d, P, r='R2').multiply_scalar(-1))
    H.add(H_c1(d, P, r='R3').multiply_scalar(-1))
    H.add(H_c2(d, P, ri='R0', rj='R1').multiply_scalar(-1))
    H.add(H_c2(d, P, ri='R2', rj='R3').multiply_scalar(-1))
    H.add(H_c3(d, P, g='CF3', ri='R0', rj='R2').multiply_scalar(-1))
    H.add(H_c3(d, P, g='CH2CH3', ri='R1', rj='R3').multiply_scalar(-1))
    return H

# print(H_all(d, P=1))

In [0]:
# Solve It ...

# try different values for P
for P in range(1,100, 10) :
    H = H_all(d, P)
    H.multiply_scalar(-1) # multiply by -1 to maximize instead of minimize

    emda_sol = emda_solver.minimize(H)
    x = emda_sol.get_minimum_energy_solution().configuration
    cnt_selected = np.count_nonzero(x)
    if not cnt_selected == 4:
        print('P={} {} -> invalid solution'.format(P, x))
    else:
        print('P={} -> valid solution: {} value: {}'.format(P, x, H.compute(x)))
        selected = [list(np.concatenate(items))[m] for m in np.nonzero(x)[0]]  
        weights = [groups[g] for g in selected]
        print("Selected molecules and their weights:\n {} {}. Total weight: {}".format(selected, weights, sum(weights)))
        break

P=1 [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1] -> invalid solution
P=11 [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1] -> invalid solution
P=21 -> valid solution: [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1] value: -131
Selected molecules and their weights:
 ['CF3', 'NH2', 'OH', 'CH2CH3'] [69, 16, 17, 29]. Total weight: 131
